Набор данных полученн в Национальном институте диабета, болезней органов пищеварения и почек. 

Цель - предскание, есть ли у пациента диабет. 

Все пациенты датасета — женщины не моложе 21 года индейского происхождения Пима

Pregnancies — количество беременностей.

Glucose — концентрация глюкозы в плазме через два часа при пероральном тесте на толерантность к глюкозе.

BloodPressure — диастолическое артериальное давление (мм рт. ст.).

SkinThickness — толщина кожной складки трицепса (мм).

Insulin — двухчасовой сывороточный инсулин (ме Ед/мл).

BMI — индекс массы тела (вес в кг/рост в м, взятый в квадрат)

DiabetesPedigreeFunction — функция родословной диабета (чем она выше, тем выше шанс наследственной заболеваемости).

Age — возраст.

Outcome — наличие диабета (0 — нет, 1 — да).

In [28]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

#Прочитаем наши данные и выведем первые пять строк таблицы
diabetes = pd.read_csv('https://drive.google.com/uc?export=download&id=17iIT9DMK1CRNTt1eHMdE1QTdyfqVPz2M')
diabetes.head()

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome,Gender
0,6,98,58,33,190,34.0,0.430,43,0,Female
1,2,112,75,32,0,35.7,0.148,21,0,Female
2,2,108,64,0,0,30.8,0.158,21,0,Female
3,8,107,80,0,0,24.6,0.856,34,0,Female
4,7,136,90,0,0,29.9,0.210,50,0,Female


In [29]:
#Найдем все повторяющиеся строки в данных и удалим их. 

mask = diabetes.duplicated()
diabetes_duplicates = diabetes[mask]
print(f'Число найденных дубликатов: {diabetes_duplicates.shape[0]}') #10

diabetes_dedupped = diabetes.drop_duplicates()
print(f'Результирующее число записей: {diabetes_dedupped.shape[0]}') #768

Число найденных дубликатов: 10
Результирующее число записей: 768


In [30]:
#найдем все неинформативные признаки в данных и избавимся от них. 
# В качестве порога информативности возьмем 0.95: 
# удалим все признаки, для которых 95 % значений повторяются или 95 % записей уникальны. 

#список неинформативных признаков
low_information_cols = [] 

#цикл по всем столбцам
for col in diabetes_dedupped.columns:
    #наибольшая относительная частота в признаке
    top_freq = diabetes_dedupped[col].value_counts(normalize=True).max()
    #доля уникальных значений от размера признака
    nunique_ratio = diabetes_dedupped[col].nunique() / diabetes_dedupped[col].count()
    # сравниваем наибольшую частоту с порогом
    if top_freq > 0.95:
        low_information_cols.append(col)
        print(f'{col}: {round(top_freq*100, 2)}% одинаковых значений')
    # сравниваем долю уникальных значений с порогом
    if nunique_ratio > 0.95:
        low_information_cols.append(col)
        print(f'{col}: {round(nunique_ratio*100, 2)}% уникальных значений')
        
#удаление неинформативного признака
clean_diabetes = diabetes_dedupped.drop(low_information_cols, axis=1)
print(f'Результирующее число признаков: {clean_diabetes.shape[1]}') #9 - осталось столбцов


Gender: 100.0% одинаковых значений
Результирующее число признаков: 9


In [31]:
#найдем пропуски в данных с помощью метода isnull()
# Так как в таблице пропуски в столбцах Glucose, BloodPressure, SkinThickness, Insulin и BMI обозначены нулём, традиционные методы поиска пропусков ничего не покажут.
# Из библиотеки numpy возьмем символ пропуска: np.nan и в столбцах в столбцах Glucose, BloodPressure, SkinThickness, Insulin и BMI заменим им все нули

display(clean_diabetes.isnull().head(5)) # все False, так как на месте пропусков стояли нули

"""
второй вариант изменения значений
list_with_nan = ['Glucose', 'BloodPressure', 'SkinThickness', 'Insulin', 'BMI']
for col in list_with_nan:
    info_diabetes[col] = info_diabetes[col].replace({0: np.nan})
print(info_diabetes)

третий вариант - функция
def nan_replacement(x):
    return np.nan if x == 0 else x
list_with_nan = ['Glucose', 'BloodPressure', 'SkinThickness', 'Insulin', 'BMI']
for col in list_with_nan:
    clean_diabetes[col] = clean_diabetes[col].apply(nan_replacement)
print(clean_diabetes)
"""

list_with_nan = ['Glucose', 'BloodPressure', 'SkinThickness', 'Insulin', 'BMI']
for col in list_with_nan:
    for x in col:
        clean_diabetes[col] = clean_diabetes[col].apply(lambda x: np.nan if x==0 else x)

nan_percent = clean_diabetes.isnull().mean().round(2).sort_values(ascending=False)
print(nan_percent)


,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
0,False,False,False,False,False,False,False,False,False
1,False,False,False,False,False,False,False,False,False
2,False,False,False,False,False,False,False,False,False
3,False,False,False,False,False,False,False,False,False
4,False,False,False,False,False,False,False,False,False


Insulin                     0.49
SkinThickness               0.30
BloodPressure               0.05
Glucose                     0.01
BMI                         0.01
Pregnancies                 0.00
DiabetesPedigreeFunction    0.00
Age                         0.00
Outcome                     0.00
dtype: float64


In [32]:
#Удалим из данных признаки, где число пропусков составляет более 30 %. 

#создим копию исходной таблицы
drop_diabetes = clean_diabetes.copy()
#зададим минимальный порог: вычисляем 70% от числа строк
thresh = drop_diabetes.shape[0]*0.7
#удаляем столбцы, в которых более 30% (100-70) пропусков
drop_diabetes = drop_diabetes.dropna(thresh=thresh, axis=1)
#отображаем результирующую долю пропусков
drop_diabetes.isnull().mean()
print(drop_diabetes.shape) 


(768, 8)


In [33]:
#Удалим из данных только те строки, в которых содержится более двух пропусков одновременно. 

thresh_1=drop_diabetes.shape[1]-2
drop_diabetes = drop_diabetes.dropna(axis=0, thresh=thresh_1)
print(drop_diabetes.shape[0]) # результирующее число записей


761


In [34]:
#В оставшихся записях заменим пропуски на медиану. 

cols_null_percent = drop_diabetes.isnull().mean() * 100
cols_with_null = cols_null_percent[cols_null_percent>0].sort_values(ascending=False)

display(cols_with_null)

#создаем копию исходной таблицы
fill_diabetes = drop_diabetes.copy()
#создаем словарь имя столбца: число(признак) на который надо заменить пропуски
values = {
    'SkinThickness': fill_diabetes['SkinThickness'].median(),
    'BloodPressure': fill_diabetes['BloodPressure'].median(),
    'Glucose': fill_diabetes['Glucose'].median(),
    'BMI': fill_diabetes['BMI'].median(),
}
#заполняем пропуски в соответствии с заявленным словарем
fill_diabetes = fill_diabetes.fillna(values)
#выводим результирующую долю пропусков
print(fill_diabetes['SkinThickness'].mean().round(1)) 

SkinThickness    28.909330
BloodPressure     3.679369
Glucose           0.657030
BMI               0.525624
dtype: float64

29.1


In [35]:
#Найдем количество выбросов с помощью классического метода межквартильного размаха в признаке SkinThickness? 

def outliers_iqr(data, feature):
    x = data[feature]
    quartile_1, quartile_3 = x.quantile(0.25), x.quantile(0.75),
    iqr = quartile_3 - quartile_1
    lower_bound = quartile_1 - (iqr * 1.5) #по умолчанию число межквартильных размахов в одну сторону=1.5
    upper_bound = quartile_3 + (iqr * 1.5)
    outliers = data[(x < lower_bound) | (x > upper_bound)]
    cleaned = data[(x >= lower_bound) & (x <= upper_bound)]
    return outliers, cleaned

# Примененам функцию к датасету и столбцу. 
outliers, cleaned = outliers_iqr(fill_diabetes, 'SkinThickness')
print(f'Число выбросов по методу Тьюки: {outliers.shape[0]}') #87 
print(f'Результирующее число записей: {cleaned.shape[0]}')#674


Число выбросов по методу Тьюки: 87
Результирующее число записей: 674


In [36]:
print(fill_diabetes[fill_diabetes['SkinThickness'] == 0])

Empty DataFrame
Columns: [Pregnancies, Glucose, BloodPressure, SkinThickness, BMI, DiabetesPedigreeFunction, Age, Outcome]
Index: []


In [37]:
#Найдем количество выбросов с помощью классического метода z-отклонения в признаке SkinThickness?

#проверим, есть ли в столбце ['SkinThickness'] показатель ноль. 
print(fill_diabetes[fill_diabetes['SkinThickness'] == 0]) #нет

def outliers_z_score(data, feature, log_scale=False):
    if log_scale:
        x = np.log(data[feature]+1)
    else:
        x = data[feature]
    mu = x.mean()
    sigma = x.std()
    lower_bound = mu - 3 * sigma
    upper_bound = mu + 3 * sigma
    outliers = data[(x < lower_bound) | (x > upper_bound)]
    cleaned = data[(x >= lower_bound) & (x <= upper_bound)]
    return outliers, cleaned

outliers, cleaned = outliers_z_score(fill_diabetes, 'SkinThickness', log_scale=False)
print(f'Число выбросов по методу z-отклонения: {outliers.shape[0]}') #4
print(f'Результирующее число записей: {cleaned.shape[0]}')#757


Empty DataFrame
Columns: [Pregnancies, Glucose, BloodPressure, SkinThickness, BMI, DiabetesPedigreeFunction, Age, Outcome]
Index: []
Число выбросов по методу z-отклонения: 4
Результирующее число записей: 757


In [38]:
#Гистограмма распределения признака DiabetesPedigreeFunction свидетельствует о логнормальном характере признака, стоит задуматься о его логарифмировании

# Найдем число выбросов в признаке DiabetesPedigreeFunction

def outliers_iqr_mod(data, feature, left = 1.5, right = 1.5, log_scale=False):
    if log_scale:
        x = np.log(data[feature])
    else:
        x= data[feature]
    quartile_1, quartile_3 = x.quantile(0.25), x.quantile(0.75),
    iqr = quartile_3 - quartile_1
    left = left * iqr
    right = right * iqr
    lower_bound = quartile_1 - left
    upper_bound = quartile_3 + right
    outliers = data[(x < lower_bound) | (x > upper_bound)]
    cleaned = data[(x >= lower_bound) & (x <= upper_bound)]
    return outliers, cleaned

outliers, cleaned = outliers_iqr_mod(fill_diabetes, 'DiabetesPedigreeFunction', left=1.5, right=1.5, log_scale=False)
print(f'Число выбросов по методу Тьюки: {outliers.shape[0]}') #29



Число выбросов по методу Тьюки: 29


In [39]:
#Затем найдем число выбросов в этом же признаке в логарифмическом масштабе 

def outliers_iqr_mod(data, feature, left = 1.5, right = 1.5, log_scale=True):
    if log_scale:
        x = np.log(data[feature])
    else:
        x= data[feature]
    quartile_1, quartile_3 = x.quantile(0.25), x.quantile(0.75),
    iqr = quartile_3 - quartile_1
    left = left * iqr
    right = right * iqr
    lower_bound = quartile_1 - left
    upper_bound = quartile_3 + right
    outliers = data[(x < lower_bound) | (x > upper_bound)]
    cleaned = data[(x >= lower_bound) & (x <= upper_bound)]
    return outliers, cleaned

outliers, cleaned = outliers_iqr_mod(fill_diabetes, 'DiabetesPedigreeFunction', left=1.5, right=1.5, log_scale=True)
print(f'Число выбросов по методу Тьюки: {outliers.shape[0]}') #0


Число выбросов по методу Тьюки: 0
